In [2]:
import requests
import lxml.html as lh
import pandas as pd
import re
import numpy as np

# Question 1:

Acccess Target Page

In [3]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)        

Transform Target content to a format that is suitable for further processing

In [4]:
doc = lh.fromstring(page.content)

The content of the rows is frames by <tr> in the html code. This structure is leveraged to gather the content of all rows 

In [5]:
tr_elements = doc.xpath('//tr')

Loop threw all rows and save the content in the two dimensional list 'list_toronto'. First row is igonred since it is a header, also rows with more than 3 elements are ignored

In [6]:
#Create empty list
list_toronto=[]

for r in range(1,len(tr_elements)): # ignore header
    row=[]
    if len(tr_elements[r])<4: # ignore any row with more than three entries
        for index, t in enumerate(tr_elements[r]):
            name=t.text_content()
            row.append((name.replace('\n','')))
    list_toronto.append(row)    

Get list with colume names

In [7]:
column_names=[]
for index, t in enumerate(tr_elements[0]):
        name=t.text_content()
        #print (index)
        column_names.append((name.replace('\n','')))
column_names

['Postal code', 'Borough', 'Neighborhood']

Create Panda Dataframe

In [8]:
df_toronto=pd.DataFrame(list_toronto)
df_toronto.columns=column_names
df_toronto.columns=df_toronto.columns.str.replace(' ', '_')
df_toronto.head()

,Postal_code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [9]:
df_toronto.tail()

,Postal_code,Borough,Neighborhood
179,M9Z,Not assigned,
180,,Canadian postal codes,
181,NLNSPENBQCONMBSKABBCNU/NTYTABCEGHJKLMNPRSTVXY,None,None
182,None,None,None
183,None,None,None


Drop all not assined postal codes

In [10]:
df_toronto=df_toronto[df_toronto.Borough!="Not assigned"]

check_format is a function that checks that the postal codes have the wright format using regular expressions

In [11]:
def check_format(plz):
    return  bool(re.match('M\d\D',plz))
check_format(df_toronto.iloc[1,0])

True

Remove rows with incorrect postal code format

In [12]:
df_toronto['Postal_code']=df_toronto['Postal_code'].astype('str') # cast all values to type string
df_toronto=df_toronto[df_toronto['Postal_code'].apply(check_format)]
df_toronto.tail()

,Postal_code,Borough,Neighborhood
160,M8X,Etobicoke,The Kingsway / Montgomery Road / Old Mill North
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,Business reply mail Processing CentrE
169,M8Y,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...
178,M8Z,Etobicoke,Mimico NW / The Queensway West / South of Bloo...


repalce dashes with commas to seperate the neighbourhoods

In [13]:
df_toronto['Neighborhood']=df_toronto['Neighborhood'].str.replace(' /', ',')
df_toronto.reset_index(drop=True, inplace=True)


Print the shape of the dataframe

In [14]:
df_shape=df_toronto.shape
"Number of Rows: {}; Number of columns: {}".format(df_shape[0],df_shape[1])

'Number of Rows: 103; Number of columns: 3'

# Question 2

Read cvs with geospatial data for Toronto

In [15]:
url="http://cocl.us/Geospatial_data"
c=pd.read_csv(url)

Merge the geospatial data with df_toronto

In [16]:
df_toronto_geo=pd.merge(df_toronto, c, left_on='Postal_code', right_on='Postal Code', sort=True)
df_toronto_geo.drop(columns='Postal Code', inplace=True)
df_toronto_geo.head(10)

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


The function get_borough_of_Toronto returns true if the word 'Toronto' is in the Borough's name else it returns false

In [17]:
def get_borough_of_Toronto(plz):
    return  bool(re.search('Toronto',plz))
get_borough_of_Toronto(df_toronto_geo.iloc[50,1])

True

 Use the function get_borough_of_Toronto to extract the Boroughs of Toronto

In [18]:
df_toronto_reduced=df_toronto_geo[df_toronto_geo['Borough'].apply(get_borough_of_Toronto)]

In [414]:
df_toronto_reduced.reset_index(drop=True, inplace=True)
df_toronto_reduced.head()

,Postal_code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


# Question 3

Cell content:

CLIENT_ID = 'xxxxxx' # my Foursquare ID 

CLIENT_SECRET = 'xxxxxxxx' # my Foursquare Secret

VERSION = '20180605' # Foursquare API version



In [426]:
# The code was removed by Watson Studio for sharing.

Funktion that connects to Foursquare API and returns nearby venues for given location data

In [21]:
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        rsp_code=500    
        while rsp_code!=200:    # checks if responds has corrected code (retry if database retruns error code)
            rr = requests.get(url)
            rsp_code=rr.json()["meta"]["code"]
            print(rsp_code)
    
        results = rr.json()["response"]['groups'][0]['items']
        
        #results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Call getNearbyVenues for all neighbourhoods of Toronto

In [22]:
toronto_venues = getNearbyVenues(names=df_toronto_reduced['Neighborhood'],
                                   latitudes=df_toronto_reduced['Latitude'],
                                   longitudes=df_toronto_reduced['Longitude']
                                  )

The Beaches
200
The Danforth West, Riverdale
200
India Bazaar, The Beaches West
200
Studio District
200
Lawrence Park
200
Davisville North
200
North Toronto West
200
Davisville
200
Moore Park, Summerhill East
200
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
200
Rosedale
200
St. James Town, Cabbagetown
200
Church and Wellesley
200
Regent Park, Harbourfront
200
Garden District, Ryerson
200
St. James Town
200
Berczy Park
200
Central Bay Street
200
Richmond, Adelaide, King
200
Harbourfront East, Union Station, Toronto Islands
200
Toronto Dominion Centre, Design Exchange
200
Commerce Court, Victoria Hotel
200
Roselawn
200
Forest Hill North & West
200
The Annex, North Midtown, Yorkville
200
University of Toronto, Harbord
200
Kensington Market, Chinatown, Grange Park
200
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
200
Stn A PO Boxes
200
First Canadian Place, Underground city
200
Christie
200
Dufferin, Dovercou

 The dataframe toronto_venues contains information of all found venues

In [27]:
print(toronto_venues.shape)
toronto_venues.head()

(1618, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


It can be seen that the number of venues varies significanlty per neighbourhood

In [363]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,62,62,62,62,62,62
Christie,18,18,18,18,18,18
Church and Wellesley,74,74,74,74,74,74
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,39,39,39,39,39,39


In [382]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(columns='Neighborhood', inplace=True)
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [205]:
toronto_onehot.shape

(1624, 232)

Since the number of vanues varies significanlty for the neighorhoods I used the number of found vanues as additional feature to classify the neighorhoods, the information can be found in the column 'count'

In [391]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped['count']=toronto_venues.groupby('Neighborhood').count().iloc[:,1].values/LIMIT
fixed_columns = [toronto_grouped.columns[0]] +[toronto_grouped.columns[-1]] + list(toronto_grouped.columns[1:-1])
toronto_grouped=toronto_grouped[fixed_columns]
toronto_grouped.head()

,Neighborhood,count,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Berczy Park,0.58,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.23,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
2,Business reply mail Processing CentrE,0.15,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.14,0.071429,0.071429,0.142857,0.142857,0.142857,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000
4,Central Bay Street,0.62,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.016129,0.0,0.0,0.0,0.0,0.0,0.016129


In [393]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2          Restaurant  0.03
3  Italian Restaurant  0.03
4            Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
                venue  freq
0                Café  0.13
1      Breakfast Spot  0.09
2         Coffee Shop  0.09
3           Nightclub  0.09
4  Italian Restaurant  0.04


----Business reply mail Processing CentrE----
                  venue  freq
0           Pizza Place  0.07
1         Auto Workshop  0.07
2               Brewery  0.07
3  Fast Food Restaurant  0.07
4               Butcher  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service  0.14
2  Airport Terminal  0.14
3           Airport  0.07
4     Boat or Ferry  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.21
1  

In [394]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [395]:
from sklearn.cluster import KMeans

kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 2, 2, 2, 0, 2, 1, 1, 0, 2, 2, 1, 2, 1, 1, 2, 2, 0, 2, 0, 2, 2,
       2, 0, 0, 1, 2, 2, 0, 1, 0, 1, 0, 2, 2, 2, 0, 1, 0], dtype=int32)

In [396]:
# !conda install -c conda-forge folium=0.5.0 --yes 



In [397]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [416]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood','count']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted[['Neighborhood','count']] = toronto_grouped[['Neighborhood','count']]

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

Merge the dataframes df_toronto_reduced (geospatial data) and neighborhoods_venues_sorted (information about venues) for illustration

In [402]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto_reduced

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal_code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,count,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,0.04,Health Food Store,Trail,Pub,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,0.43,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Ice Cream Shop,Pub,Pizza Place,Lounge,Liquor Store
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,0.17,Fast Food Restaurant,Movie Theater,Steakhouse,Sushi Restaurant,Brewery,Fish & Chips Shop,Pub,Restaurant,Italian Restaurant,Ice Cream Shop
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,0.40,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,0.04,Dim Sum Restaurant,Park,Bus Line,Swim School,Yoga Studio,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


Visualize cluster output on Toronto map

In [411]:
latitude=43.651070
longitude=-79.347015
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [419]:
toronto_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)

To get an better understanding of the cluster results, cluster_venues_sorted consits of the 10 most common vanues for each cluster as well as the average number of venues per neighbouerhood

In [423]:
toronto_grouped_by_clustering=toronto_grouped_clustering.groupby('Cluster Labels').mean().reset_index()
toronto_grouped_by_clustering.head()
columns=[]
for ind in np.arange(num_top_venues):
        columns.append('{}th Most Common Venue'.format(ind+1))
        
cluster_venues_sorted = pd.DataFrame(columns=columns) 

for ind in np.arange(toronto_grouped_by_clustering.shape[0]):
    cluster_venues_sorted.loc[ind] = return_most_common_venues(toronto_grouped_by_clustering.iloc[ind, :], 10)
cluster_venues_sorted['Ave. Nr. of Venues']=toronto_grouped_by_clustering['count']*LIMIT
cluster_venues_sorted

,1th Most Common Venue,2th Most Common Venue,3th Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Ave. Nr. of Venues
0,Coffee Shop,Café,Italian Restaurant,Bakery,Restaurant,Bar,Pizza Place,Greek Restaurant,Sandwich Place,Park,45.333333
1,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Italian Restaurant,American Restaurant,Seafood Restaurant,Gym,Clothing Store,94.222222
2,Park,Trail,Café,Music Venue,Playground,Garden,Coffee Shop,Pub,Restaurant,Mexican Restaurant,12.555556


## Description Cluster 0:

Neighbourhoods in Cluster 0 are dominated by food related venues such as coffee shops 
The density of venues is intermidate with an average of 45 per neighourhood

## Description Cluster 1:

Neighbourhoods in Cluster 1 are also dominated by food related venues such as coffee shops. However, Hotels are the thrid most common venue in this cluster. This can be explained by the fact that the neighouborhoods in cluster 1 are in approximity to the harbour and therefore offer a great view. 
The density of venues is high most of the neighouborhood reach the query limit of 100 venues. 

## Description Cluster 2:

Neighbourhoods in Cluster 2 are dominated by parks and trails. The density of venues is low with an average of only 12. Further the neighourhoods in cluster 2 have in average the largest distance to the harbour.